<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex15notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data2022 ex15notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/Data-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2022


----
## 多変量解析や機械学習に関するデモ
----

---
### 準備



以下，コードセルを上から順に実行していってね．

In [ ]:
# 科学技術計算のライブラリ NumPy のモジュールを np という名前で使えるようにする
import numpy as np

# グラフを描くためのライブラリ matplotlib の pyplot を plt という名前でインポート
import matplotlib.pyplot as plt

# 機械学習のためのライブラリ scikit-learn のいろいろ
from sklearn.cluster import KMeans                  # K-means 法のクラス
from sklearn.preprocessing import StandardScaler    # 標準化の手続き
from sklearn.linear_model import LogisticRegression # ロジスティック回帰
from sklearn.neural_network import MLPClassifier    # 階層型ニューラルネット

# 「1000種類の物体を識別するニューラルネットを動かしてみよう」のためのほげ
import pickle
import torch
from torchvision import models, transforms
from PIL import Image
import json

In [ ]:
#####  データの最初の nx x ny 枚を可視化する関数の定義
#
def mosaicImage(ax, dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype=int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))

    # 表示
    ax.axis('off')
    ax.imshow(img, cmap='gray')


----
### クラスター分析／クラスタリング

**クラスター分析**／**クラスタリング** は，多変量データをクラスタ分けする（似たもの同士でグループに分ける）ための手法です．

ここでは，「K-means法（K-平均法）」と呼ばれるアルゴリズムを使って，猫の画像をクラスタリングしてみましょう．まずは，データやプログラムの準備をします．以下のセルたちを順次実行してください．



(1) 猫画像データを入手して，一部を表示させます．

In [ ]:
# 猫顔画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/cat131.npz
cat = np.load('cat131.npz')['cat131']
Ncat, Dcat = cat.shape
print(f'データ数 x 次元数 = {Ncat} x {Dcat}')

# 最初の 20 枚を表示
fig, ax = plt.subplots(figsize=(6, 6))
mosaicImage(ax, cat, 5, 4)
plt.show()  

(2) K-means法によってクラスタリングを行います．K-means法は，データとデータの間の距離にもとづいて，データを K 通りのクラスタに分ける手法です（クラスタの数 K はあらかじめ指定します）．ここでは，Python で機械学習に関する様々な処理を行える scikit-learn というライブラリを使っています．

- [scikit-learn](https://scikit-learn.org/)
- [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
K = 3  # K-means 法でいくつのクラスタに分けるか

model = KMeans(n_clusters=K) # 学習モデルの定義． n_cluster でクラスタ数を指定
model.fit(cat) # K-means クラスタリングを実行
label = model.predict(cat) # 各画像の所属クラスタを求める
print(label)

(3) 結果を表示させてみましょう．一つのクラスタに属する画像を横に並べてあります．

In [ ]:
# 各クラスタに割り振られた画像のうち最初の10枚を横に並べて表示
nc = 10
dat = np.zeros((K, nc, Dcat))
for ik in range(K):
    idx = label == ik
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = cat[idx, :]
    else:
        dat[ik, :, :] = cat[idx, :][:nc, :]

dat = dat.reshape((-1, Dcat))

fig, ax = plt.subplots(figsize=(12, 12))
mosaicImage(ax, dat, nc, K)
plt.show()  

**★★★ 結果を観察しよう＆やってみよう★★★**

上記では，猫画像を K = 3 の K-means法でクラスタリングする実験を行っています．

K をいろいろ変えて（(2)のセルの `K = 3` の `3` を変えて）(2)と(3)の2つのコードセルを実行し直して，結果を観察しよう．

ただし，データ数がそれほど多くないので，Kをあまり大きくすると空のクラスタができたりしてエラーになるかもしれません．また，K-meansクラスタリングの結果は初期値に依存するので，実行するたびに結果が変わります．

----
### ロジスティック回帰やニューラルネットによる手書き数字画像の識別

手書き数字画像のデータを使い，画像に描かれた数がいくつであるかを推測する実験をやってみましょう．


#### データの準備

データを入手して，一部を画像として眺めてみます．

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/minimnist.npz
rv = np.load('minimnist.npz')
X_train = rv['datL'].astype(float)
Y_train = rv['labL']
X_test = rv['datT'].astype(float)
Y_test = rv['labT']
#print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

K = 10 # クラス数

# データの各要素を標準化（平均 0 分散1 にスケーリング）
scaler = StandardScaler()
XX_train = scaler.fit_transform(X_train)
XX_test  = scaler.transform(X_test)

print('### 手書き数字画像 ###')
print(f'データ数 x 次元数 = {X_train.shape[0]} x {X_train.shape[1]}')

# 最初の 60 枚を表示
fig, ax = plt.subplots(figsize=(12, 6))
mosaicImage(ax, X_train, 15, 4, nrow=28, ncol=28)
plt.show()

for i in range(4):
    print(Y_train[i*15:(i+1)*15])

このデータセットでは，1つの手書き数字画像は 28 x 28 画素ありますので，1つのデータを $784 (=28\times 28)$ 次元ベクトルとして扱うことになります．
画像の下に並んだ数字 `0 9 3 9 3 ...` は，これらの画像の手書き数字に対応する実際の数（正解）を表しています．

#### ロジスティック回帰の実験

前回ちょこっと説明した「ロジスティック回帰」は，与えられたデータを2通りに分類するものでしたが，よりたくさんの分類ができるように拡張できます．ここでは，784次元ベクトル5,000個から成る手書き数字画像データを，0 から 9 の10通りに分類させてみましょう．

ロジスティック回帰モデルの学習では，間違えたらパラメータを修正することを何回も繰り返す必要があります．ここでは 500 回学習させてみます．少し時間がかかります．

In [ ]:
# ロジスティック回帰モデルのインスタンスを生成
model = LogisticRegression(max_iter=500)

# 学習データを用いてモデルのパラメータを推定（学習）
model.fit(XX_train, Y_train)

学習したモデルに数を予測させて，何個正解したか求めてみると，こんなんなります．

In [ ]:
# 学習データのクラスを予測させてみる
Yt_train = model.predict(XX_train)
nc_train = np.sum(Yt_train == Y_train) # 正解数

# テストデータのクラスを予測させてみる
Yt_test = model.predict(XX_test)
nc_test = np.sum(Yt_test == Y_test) # 正解数

# 結果を表示
print(f'学習データ: {nc_train}/{len(Y_train)} = {nc_train/len(Y_train)}')
print(f'テストデータ: {nc_test}/{len(Y_test)} = {nc_test/len(Y_test)}')

テストデータの一部を画像として見るとこんなん．いくつか間違えてますね．

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
mosaicImage(ax, X_test, 15, 4, nrow=28, ncol=28)
plt.show()
print()
print('予測:')
for i in range(4):
    print(Yt_test[i*15:(i+1)*15])
print()
print('正解:')
for i in range(4):
    print(Y_test[i*15:(i+1)*15])

#### ニューラルネットの実験

次は，「ニューラルネットワーク」です．これは，最近のいわゆる人工知能の中核となっている機械学習の仕組みです（「深層学習 (deep learning)」という言葉を聞いたことがあるひともいるかもしれません）．
以下では，簡単なニューラルネットの例で実験できるようにしてあります．

ここで使っているニューラルネットも学習の繰り返しが必要です．次のセルを実行すると，学習データ 5000 個を使って学習します． Iteration は1回の学習（5000個のデータ全部を使ってパラメータを推定する処理）を表し，loss はその学習を終えた時点での「損失」（正解とのずれの指標，小さいほど良い）の値を表します．

繰り返すごとに損失が小さくなり，学習が進んでいくのがわかるでしょう．

In [ ]:
# 階層型ニューラルネットのインスタンスを生成
model = MLPClassifier(hidden_layer_sizes=(200, 200,), activation='relu', verbose=1)
print(model)

# 学習データを用いてモデルのパラメータを調整（学習）
model.fit(XX_train, Y_train)

学習したモデルに数を予測させて，何個正解したか求めてみると，こんなんなります．

In [ ]:
# 学習データのクラスを予測させてみる
Yt_train = model.predict(XX_train)
nc_train = np.sum(Yt_train == Y_train) # 正解数

# テストデータのクラスを予測させてみる
Yt_test = model.predict(XX_test)
nc_test = np.sum(Yt_test == Y_test) # 正解数

# 結果を表示
print(f'学習データ: {nc_train}/{len(Y_train)} = {nc_train/len(Y_train)}')
print(f'テストデータ: {nc_test}/{len(Y_test)} = {nc_test/len(Y_test)}')     

**★★★ やってみよう★★★**

1. ロジスティック回帰による手書き数字識別の実験をやって結果を観察しよう．正解率は何%くらい？
1. ニューラルネットによる手書き数字識別の実験をやって結果を観察しよう．正解率は何%くらい？



---
### 1000種類の物体を識別するニューラルネットを動かしてみよう

ニューラルネットワークを用いた画像識別の実験をやってみましょう． いろんなものが識別できた方が楽しいと思いますので，1000種類の物体を識別させてみます． ただし，そういうことができるニューラルネットワークを一から学習させようとすると大変（↑ののりで実行したら1週間以上かかかります）ですので，ここではすでに学習済みのものを利用することにします．

#### 準備

まずは実験に使う画像データを準備します．

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/animalphoto.pickle
with open('animalphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for d in hoge:
    img = Image.frombytes(**d)
    imgList.append(img)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

1000個あるカテゴリの番号とそのラベルの対応表を作成します．

In [ ]:
# ImageNet カテゴリラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300: 
        print(ik, labels[ik])

上記の出力は，1000カテゴリのうちの一部のものの名前を表します．「ネコ」みたいなものも1カテゴリではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

学習済みのニューラルネットワークのパラメータを入手します．
規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.eval()
print(vgg16)

上記のセルを実行すると，次のことが行われます．
1. ニューラルネットワークの学習モデルを作成する．
1. 学習済みのネットワークパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

ここで用いているのは VGG16 というニューラルネットワークモデルです．イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットワークモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットワークモデルです．

ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた画像約120万枚を使って画像識別のコンペが行われました．これらの画像には1000種類の物体のいずれか一つが写っており，何が写っているのかを表す情報も一緒に与えられていました．
VGG16のニューラルネットワークは，そのような120万枚の画像を与えられて，それらの画像に写っているものが何であるか正しく識別するように訓練されています．訓練（学習）した1000種類の中のどれであるかを答えることしかできませんので，それ以外の画像を見せても，1000種類の中のどれかに間違えます．

画像をネットワークに入力できる形式に加工する処理を定義しておきます．


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#### ★★★ やってみようその1

上記のサンプル画像たちを VGG16 に識別させてみましょう．


In [ ]:
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のカテゴリの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とカテゴリラベルを表します．
出力の値は，そのカテゴリと判定することの「確信度」と解釈できます．

#### ★★★ やってみようその2

自分で用意した画像でも実験してみましょう．

- 上記では動物の例ばかりですが，1000のカテゴリの中にはそれ以外にも様々なものがあります（「人間」というカテゴリはありません．どうなる？）
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



(1) まずは，ウェブ等で適当な画像を探して，この Colab notebook を実行しているコンピュータへ保存しましょう．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルを実行して，ファイルをアップロードします．

In [ ]:
# Colab へファイルをアップロード
try:
    from google.colab import files
    rv = files.upload()
except:
    print('このコードは Colab 以外の環境では実行できないよ．')

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
myimg = Image.open('hoge.png') # ファイル名を変更しましょう
if myimg.mode != 'RGB':
    myimg = myimg.convert('RGB')
fig, ax = plt.subplots(1)
ax.imshow(myimg)
ax.axis('off')
plt.show()

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')